# Randomness and Markov Chains

In this notebook, we will study random numbers, and one use: random processes in the form of Markov Chains. 

The function `randu` was a *linear congruential generator*.  The random sequence is generated as 
$$ I_{j+1} = (a I_j + c) \pmod m.$$
This is implemented as a Python class below. You should run this cell.

In [7]:
class badrand (object) :
    """An implementation of a BAD random number generator.  This is based on
    randu() which appeared in the IBM library.  It should NEVER be used and
    will warn you to not use it!"""

    def __init__ (self, seed) :
        """Initialize the random number generator.  You MUST provide a seed.
        This will warn you about using a bad random number generator."""
        self.seed = seed
        self.m = 2**31
        self.minv = 1.0/self.m
        self.a = 65539
        self.c = 0
        print("Congratulations, you just initialized a BAD random number generator.\n"
              +"You are now ready to NOT generate good random numbers.")

    def ran (self, N) :
        """Generate N random numbers in the interval [0,1).  This will warn
        you that you have bad random numbers."""
        res = np.zeros(N)
        for j in range(N) :
            self.seed = (self.a*self.seed+self.c)%self.m
            res[j] = self.seed * self.minv
        print("Bad random numbers generated. "
              +"You are not using these for anything serious, right?")
        return res

To generate $N$ random numbers, you should use code similar to what follows. The name randu is arbitrary.  The seed must be a positive integer.  
```
randu = badrand(seed)
# To get N random numbers we then use
br = randu.ran(N)
```

For our analysis we will use $N=300000$. Generate $N$ random numbers from a good random number generator, `np.random.rand` for example, and $N$ from the bad random number generator.  To see that `badrand` is not trivially bad, create a histogram of values generated for both sets of random numbers. (Note: When using `matplotlib.pyplot.hist` you should probably plot the histograms as steps, not bars; see the `histtype='step'` option.  Also, the histograms may not appear to be flat when you make them.  If so, check the range of values on the y-axis, ....)

In [8]:
import numpy as np
import matplotlib.pyplot as plt

## Your code here...

The behavior looks perfectly sensible, so why not use this random number generator?  Let us jump to the problem: the random numbers are correlated. One easy way to see this is to perform a calculation using every 3rd random number generated. Recall that you can select every 3th element in array by specifying a step: calling `br[0::3]` will access every 3rd element beginning with the 0th one. Define $x$ to contain every third element starting with the 0th one, $y$ starting with the 1st one, and $z$ starting with the 2nd. Finally define $i = 9x-6y+z$.

Calculate $i$ for both the good and bad random numbers.  Produce a nice figure showing $i$ versus $x$ for both sets of random numbers.  Using the period marker, ‘.’, for these plots is recommended.  Plot the good random numbers first.  You should find a band that is sampled fairly evenly.  Next plot the bad random numbers.

In [9]:
## Your code here...

## Probability Density Transformations

Another useful concept involves being able to draw random numbers from complicated probability distributions. Generally, transforming one probability distribution to another is a type of [integral transformation](https://en.wikipedia.org/wiki/Probability_integral_transform). This type of transformation can appear in a number of settings, ranging from statistical analyses, to improving integration efficiency in Monte Carlo integration methods.

### A simple example

Here we will examine a simple transformation between a uniform and linear probability distribution. The uniform probability distribution on the interval $[0,1)$ is given by

$$
p_{\rm uni}(x) = \begin{cases}
1 & 0 \le x < 1  \\
0 & {\rm otherwise}
\end{cases}
$$

while the linear distribution is given by

$$
p_{\rm lin}(y) = \begin{cases}
y & 0 \le y < \sqrt{2}\\
0 & {\rm otherwise}
\end{cases}\,.
$$

The coordinate transform required to transform between these distributions is also simple, $ y = \sqrt{2x}$. Below, generate $300000$ random numbers on the interval $[0, 1)$ using `np.random.rand`. Produce a histogram as above. This time, because we are interested in evaluating this as a probability distribution, you should make sure to supply the `density='true'` option to `hist` in order to correctly normalize the distribution. Last, apply the coordinate transformation, and plot the distribution function of the transformed numbers.

## Markov Chains

As a simple model of spring weather in St. Louis, let us consider a 3-state system with the states representing a "sunny" day, a "rainy" day, and a "cloudy" day. A more realistic model may include knowledge of weather at other locations, temperatures humidity; or a full-blown climate simulation. However for a simple example we will ignore the details. Let the probabilities of transitioning from one type of weather to another be given as in the following diagram:

![Weather States](https://sites.wustl.edu/mertens/files/2021/04/weather_states.png)

Recall the transition matrix is given by $P$,

$$
\begin{pmatrix}
q_{1}^{f}\\
q_{2}^{f}\\
q_{3}^{f}
\end{pmatrix}
=
\begin{pmatrix}
P_{1\rightarrow1} & P_{2\rightarrow1} & P_{3\rightarrow1}\\
P_{1\rightarrow2} & P_{2\rightarrow2} & P_{3\rightarrow2}\\
P_{1\rightarrow3} & P_{2\rightarrow3} & P_{3\rightarrow3}
\end{pmatrix} 
\begin{pmatrix}
q_{1}\\
q_{2}\\
q_{3}
\end{pmatrix}.
$$

Write the transition matrix $\mathsf{P}$ that describes this diagram for the state vector
$$\vec{q} = \begin{pmatrix} q_☼ \\ q_☂ \\ q_☁ \end{pmatrix}.$$
Verify that the columns of this matrix add up to one.

We can use our simple model to forecast the weather.  Given an initial state representing the weather today we can predict the probabilities of it being sunny, rainy, or cloudy after 1 day, 2 days, or 1 week.  To begin pick an initial state vector representing the weather today
$$ \vec{q}^i = \left( \begin{array}{c} q_☼ \\ q_☂ \\ q_☁ \end{array} \right). $$
You can pick one that is somewhat representative of the weather right now.

In order to calculate the weather after one, two, and seven days you will need to calculate $\mathsf{P} \vec{q}^i$, $\mathsf{P}^2 \vec{q}^i$, and $\mathsf{P}^7 \vec{q}^i$. There are multiple ways to raise a matrix to a power. One way is to use the eigenvalue decomposition. As an alternative, NumPy provides a function for doing this: `np.linalg.matrix_power`.  Either of these methods is generally better than repeatedly multiplying the matrix with itself, although in our case that may the be most simple.

Print the probabilities for the state of the weather after 1, 2, and 7 days.  Note that after 7 days the result should look very much like the stationary state we found above.  For fun you may want to compare your predictions to those from a professional weather service and/or keep track of you predictions and see how they turn out.